<a href="https://colab.research.google.com/github/zwubbena/cartesian-product/blob/main/cartesian_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ============================================================================
# Program:    create_cartesian.py (Google Colab)
# Purpose:    Create all combinations of determination level values
#             from one year to the next year using a Cartesian product.
# ============================================================================

import pandas as pd

# ===============================================================
# SECTION 1: Dataset #1 - 2019 Determination Levels (DL2019)
# ===============================================================
data_2019 = {
    'DL': ['DL0', 'DL1', 'DL2', 'DL3', 'DL4'],
    'DL2019': [0, 1, 2, 3, 4]
}
a_table = pd.DataFrame(data_2019)

# ===============================================================
# SECTION 2: Dataset #2 - 2020 Determination Levels (DL2020)
# ===============================================================
data_2020 = {
    'DL': ['DL1', 'DL2', 'DL3', 'DL4'],
    'DL2020': [1, 2, 3, 4]
}
b_table = pd.DataFrame(data_2020)

# ===============================================================
# SECTION 3: Generate Cartesian Product of DL2019 × DL2020
# ===============================================================
# Perform Cartesian product using a dummy key
a_table['_key'] = 1
b_table['_key'] = 1
merged = pd.merge(a_table, b_table, on='_key')
cartesian_dl = merged.drop(columns=['_key', 'DL_x', 'DL_y'])

# ===============================================================
# Display and Export Result
# ===============================================================
from datetime import datetime
from google.colab import files

print("▶ Cartesian Product of 2019 and 2020 DLs:")
display(cartesian_dl)

# Export to Excel
today_str = datetime.today().strftime("%Y%m%d")
output_filename = f"{today_str}_Cartesian_DL_Combos.xlsx"
cartesian_dl.to_excel(output_filename, index=False)

# Offer download in Colab
files.download(output_filename)

# ============================================================================
# Python Program End
# ============================================================================

▶ Cartesian Product of 2019 and 2020 DLs:


,DL2019,DL2020
0,0,1
1,0,2
2,0,3
3,0,4
4,1,1
5,1,2
6,1,3
7,1,4
8,2,1
9,2,2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>